In [ ]:
from init_notebook import *
from src.algo.sdf.two_d.util import *
from src.algo.sdf.two_d import *

In [ ]:
def render(obj: Object, shape=(128, 128)):
    d = obj.render_distance(shape)
    inside = -d.clip(-0.01, .0) / .01
    stripes = (d * 10.) % 1. 
    stripes = np.pow(np.abs(stripes - .5) / .5, 4.)
    
    image = inside[..., None].repeat(3, 2) * .5
    image[..., 0] = stripes
    display(VF.to_pil_image(image))


In [ ]:
obj = Xor(
    #Plane((1, 0)).translate((.5, 0)),
    Circle(.3).translate((.5, .5)),
    Circle(.2).translate((.5, .8)),
    Circle(.1).translate((.7, .7)),
    Circle(.1).translate((.8, .7)),
    #radius=.2,
)#.invert()
render(obj)

In [ ]:

obj = Union(
    Circle(.5).scale(.5),
)
render(obj)

In [ ]:
render(
    Box((.3, .3)).rotate(45).subtract(
        Box((.2, .2)).subtract(
            Box((.1618, .03)), #smooth=.3
        ),
    ).translate((.5, .5))
)

In [ ]:
render(
    Box(.3).translate((.5, .5)).sin_warp(freq=4, amount=0.02, phase=(0, .5))
)

In [ ]:
n = calc_light(
    Union(
        Box(0.15),
        Circle(0.15).translate(.3),
    ).translate(.5)
    #.render_distance_normal()
    .render_mask_normal(radius=.2)
)
#n = n.repeat(2, -1)[..., 1:]
#n[..., 2] = 0
print(n.shape)
display(VF.to_pil_image(n*.5+.5))
#n

In [ ]:
class Tiles:
    L = 1
    T = 1 << 1
    R = 1 << 2
    B = 1 << 3

    TL = 1 << 4
    TR = 1 << 5
    BR = 1 << 6
    BL = 1 << 7

    ALL_EDGES = L|R|T|B
    ALL_CORNERS = TL|TR|BL|BR

    # 4x4
    EDGE_ORDER = [
        0, R, L|R, L,
        B, B|R, B|L|R, B|L,
        T|B, T|B|R, T|B|L|R, T|B|L,
        T, T|R, T|L|R, T|L,
    ]

    # 4x4
    CORNER_ORDER = [
        0, TR, TL|BR, BL,
        BR, BL|BR, BL|BR|TR, BL|TL,
        BL|TR, TL|TR|BR, TL|TR|BL|BR, TL|BL|BR,
        TL, TR|BR, TL|TR|BL, TL|TR, 
    ]

    # 7x7
    EDGE_AND_CORNER_ORDER = [
        TL|L|BL|T|B|TR|R|BR, TL|T|TR|L|BL|B|BR, TL|T|TR|BL, TL|T|TR, TL|T|TR|BR, TL|T|TR|R|BR|BL, TL|L|BL|T|B|TR|R|BR,
        BL|L|TL|T|TR|R|BR, BL|L|TL|T|TR|BR, TL|BL|BR, BL, TR|R|BR, TL|L|BL|TR|BR, TL|T|TR|R|BR|B|BL,
        TL|L|BL|TR, TL|TR|BR, BL|TL|TR|BR, TL|BL|B|BR, BL|TR, TL|TR, TL|T|TR|R|BR,  
        TL|L|BL, TR, TL|TR|R|BR, BL|L|TL|T|TR, TL, BR, BL|TR|R|BR,
        TL|L|BL|BR, BL|B|BR, BL|BR|TR, TL|BL, 0, TR|BR, TL|BL|TR|R|BR,
        TL|L|BL|B|BR|TR, TL|T|TR|BL|BR, TL|TR|BL, TL|BR, BL|BR, BL|B|BR|R|TR, TL|L|BL|TR|R|BR,
        TL|L|BL|T|B|TR|R|BR, TL|L|BL|B|BR|R|TR, TL|L|BL|B|BR, BL|B|BR|TR, TL|TR|BL|B|BR, TL|T|TR|BL|B|BR, TL|BL|B|BR|R|TR, 
    ]

    EO2 = [
        0, R, L|R|B|BR, L|R|BL|B|BR, L|R|BL|B, L|B, 0, 
        B, R|B, T|TR|L|R|B, TL|T|TR|L|R|B|BR, TL|T|L|BL|B, T|R|B, L, 
        T|R|B|BR, T|L|R|BL|B, T|L|R|B, T|TR|L|R, TL|T|L|R|B|BR, T|L|R|BL|B|BR, L|BL|B, 
        T|TR|R|B|BR, TL|T|L|R|BL|B|BR, T|L|BL|B, R|B|BR, T|TR|L|R|BL|B|BR, TL|T|TR|L|R|BL|B, TL|T|L|B, 
        T|TR|R|B, TL|T|TR|L|R, TL|T|L|R|B, T|TR|L|R|B|BR, TL|T|TR|L|R|BL|B|BR, TL|T|L|R|BL|B, T|L|B, 
        T|R, L|R|B, T|L|R|B|BR, T|TR|L|R|BL|B, TL|T|TR|L|R|B, TL|T|L, T|B, 
        0, T, T|TR|R, TL|T|L|R, T|L|R, L|R, T|L,
    ]
    def __init__(self, r1=.5):
        self.r1 = r1
    
    def render_distance(self, index: int, shape=(128, 128), offset=(0, 0), warp=1.):
        objects = []
        if index & self.ALL_EDGES == self.ALL_EDGES:
            objects.append(Box((2, 2)))
        if index & self.L:
            objects.append(Box((1, self.r1)).translate((.5, 0)))
        if index & self.R:
            objects.append(Box((1, self.r1)).translate((.5, 1)))
        if index & self.T:
            objects.append(Box((self.r1, 1)).translate((0, .5)))
        if index & self.B:
            objects.append(Box((self.r1, 1)).translate((1, .5)))
        #box = Box((self.r1*.707106, self.r1*.707106)).rotate(45)
        box = Circle(self.r1)
        if index & self.TL:
            objects.append(box)
        if index & self.TR:
            objects.append(box.translate((0, 1)))
        if index & self.BL:
            objects.append(box.translate((1, 0)))
        if index & self.BR:
            objects.append(box.translate((1, 1)))
        if index & (self.T|self.L) == (self.T|self.L):
            objects.append(Circle(1-self.r1).invert().translate((1, 1)))
        if index & (self.T|self.R) == (self.T|self.R):
            objects.append(Circle(1-self.r1).invert().translate((1, 0)))
        if index & (self.B|self.L) == (self.B|self.L):
            objects.append(Circle(1-self.r1).invert().translate((0, 1)))
        if index & (self.B|self.R) == (self.B|self.R):
            objects.append(Circle(1-self.r1).invert().translate((0, 0)))
        return (
            Union(*objects).translate(offset)
            .sin_warp(.02*warp, freq=3, phase=(0., .5))
            .sin_warp(.01*warp, freq=5, phase=(0.25, 0))
            .render_distance(shape)
        )
    def render(self, index: int, smooth=0.01, shape=(128, 128), offset=(0, 0), **kwargs):
        d = self.render_distance(index, shape=shape, offset=offset, **kwargs)
        inside = -d.clip(-smooth, .0) / smooth
        #inside = inside*inside
        stripes = (d * 10.) % 1. 
        stripes = np.pow(np.abs(stripes - .5) / .5, 4.)
        
        image = inside[..., None].repeat(3, 2) 
        #image[..., 0] = stripes
        return image

    def render_pil(*args, **kwargs):
        return VF.to_pil_image(self.render(*args, **kwargs))
    
tiles = Tiles(r1=.33)
#tiles.render(tiles.TL|tiles.TR|tiles.BL|tiles.BR)
images = []
S = 0.1
for i in tiles.EDGE_AND_CORNER_ORDER:
    edges = tiles.render(i & tiles.ALL_EDGES, smooth=S)
    corners = tiles.render(i & tiles.ALL_CORNERS, smooth=S)
    images.append(np.maximum(edges, corners))
VF.to_pil_image(make_grid([VF.to_tensor(i) for i in images], nrow=7, padding=1))

In [ ]:
sometiles = VF.to_tensor(PIL.Image.open(
    #"/home/bergi/prog/python/github/jnr/assets/texture/tileset-8x8-16x16.png"
    "/home/bergi/prog/data/game-art/overworld_tileset_grass.png"
))
sometiles = sometiles.numpy().transpose(1, 2, 0)
t1 = (0, 0)
t2 = (0, 0)#(5, 1)
t3 = (11, 3)
t1 = sometiles[t1[0]*16:(t1[0]+1)*16, t1[1]*16:(t1[1]+1)*16]
t2 = sometiles[t2[0]*16:(t2[0]+1)*16, t2[1]*16:(t2[1]+1)*16]
t3 = sometiles[t3[0]*16:(t3[0]+1)*16, t3[1]*16:(t3[1]+1)*16]
#t3 = np.ones_like(t1)
#t3[..., :3] = (0.5, .4, 0)
tiles = Tiles(r1=.4)
images = []
def render_mask(i, offset=(0, 0), smooth=.2, warp=2.):
    edges = tiles.render(i & tiles.ALL_EDGES, smooth=smooth, shape=(16, 16), offset=offset, warp=warp)
    corners = tiles.render(i & tiles.ALL_CORNERS, smooth=smooth, shape=(16, 16), offset=offset, warp=warp)
    mask = np.maximum(edges, corners)[..., :1]
    mask = np.pow(mask, 2)
    return mask

def render_norm(i, o=1/32, o2=(0, 0)):
    n = np.concat([
        (render_mask(i, offset=(o+o2[0], 0+o2[1])) - render_mask(i, offset=(-o+o2[0],  0+o2[1]))),
        (render_mask(i, offset=(0+o2[0], o+o2[1])) - render_mask(i, offset=( 0+o2[0], -o+o2[1]))),
    ], axis=-1)
    m = magnitude(n)
    mask = m > 0
    n[mask] = n[mask] / m[mask][..., None]
    return n

def quant(x, steps: float):
    return np.round(x * steps) / steps
    
for i in tiles.EDGE_AND_CORNER_ORDER:
    mask = render_mask(i, smooth=.16)
    norms = []
    amt_sum = 0
    for amt, o, o2 in (
        (1, 1/32, (-.1, 0)),
        (1, 1/8, (0, 0)),
        (1, 1/4, (-.15, 0)),
    ):
        norms.append(render_norm(i, o=o, o2=o2) * amt)
        amt_sum += amt
    norm = np.stack(norms).sum(axis=0) / amt_sum
    light_norm = vec2_to_numpy((1, 1.4))
    light_norm /= magnitude(light_norm)
    #light_norm = light_norm.repeat(mask.shape[0], 0).repeat(mask.shape[1], 1)
    light = (light_norm * norm).sum(-1, keepdims=True)
    light = light * .5 + .5
    #light = quant(light, 4)
    img = mix(t2, t1, mask)

    n = render_norm(i, o=1/10)
    ln = vec2_to_numpy((1, 0))
    ln /= magnitude(light_norm)
    l = (ln * n).sum(-1, keepdims=True)
    img = mix(img, t3, np.clip(-l, 0, 1))
    #img = t1 * mask + t2 * (1. - mask)
    #img = norm.repeat(2, -1)[..., :3] * .5 + .5
    #img = np.abs(img) * 100
    img[..., :3] *= (light * .5 + .75)
    #img = light
    images.append(np.clip(img, 0, 1))
VF.to_pil_image(resize(make_grid([VF.to_tensor(i) for i in images], nrow=7, padding=0), 1))

In [ ]:
for i, idx in enumerate(tiles.EDGE_AND_CORNER_ORDER):
    idx = idx ^ 255
    keys = []
    for k in ("TL", "T", "TR", "L", "R", "BL", "B", "BR"):
        if idx & getattr(tiles, k):
            keys.append(k)
    key = 0 if not keys else "|".join(keys)
    print(f"{key}, ", end="")
    if i % 7 == 6:
        print()